In [4]:
from bs4 import BeautifulSoup
import  requests

url = 'http://www.fedsdatacenter.com/federal-pay-rates/index.php'

#soup = BeautifulSoup(url)

r = requests.get(url)

soup = BeautifulSoup(r.text, "lxml")

#t = soup.find_all("table", class_="display Tables")

print(soup.find("table",{"class": "display"}))

<table class="display table table-striped table-bordered small" id="table-example" width="100%">
<thead>
<tr>
<th>Name</th>
<th>Grade</th>
<th>Pay Plan</th>
<th>Salary</th>
<th>Bonus</th>
<th>Agency</th>
<th>Location</th>
<th>Occupation</th>
<th>FY</th>
</tr>
</thead>
<tbody>
</tbody>
</table>


In [ ]:
from pandas.io.html import read_html
from selenium import webdriver


driver = webdriver.Firefox()
driver.get('http://www.fedsdatacenter.com/federal-pay-rates/index.php')

table = driver.find_element_by_xpath('//div[@class="display Tables"]/table//table/..')
table_html = table.get_attribute('outerHTML')

df = read_html(table_html)[0]
print df

driver.close()

In [21]:
"""
get max record count
"""
from jinja2 import Template
import requests
import json

num_display = 100

base_url = Template(
                    'http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s'
                    'Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i'
                    'DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1'
                    '=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&'
                    'mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&'
                    'bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true'
                    '&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082'
                    )

# base_url = Template("""
#                     http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s
#                     Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i
#                     DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1
#                     =true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&
#                     mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&
#                     bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true
#                     &iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082
#                     """.replace('\n', ''))

never_hit_record_count = 1000000000

u = base_url.render(next_record_start=never_hit_record_count, num_display=num_display)

print u

j = requests.get(u)

print j.json()

get_count = json.loads(j.text)
get_max_record_count = get_count['iTotalDisplayRecords']
print get_max_record_count
print int(get_max_record_count) / 100


http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&sEcho=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart=1000000000&iDisplayLength=100&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082
{u'aaData': [], u'iTotalRecords': u'8620477', u'sEcho': 1, u'iTotalDisplayRecords': u'1329915'}
1329915
13299


In [1]:
"""
Test data manipulation
"""
from jinja2 import Template
import requests
import json

num_display = 10

base_url = Template(
                    'http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&s'
                    'Echo=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart={{next_record_start}}&i'
                    'DisplayLength={{num_display}}&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1'
                    '=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&'
                    'mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&'
                    'bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true'
                    '&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082'
                    )

u = base_url.render(next_record_start=0, num_display=num_display)

print u

j = requests.get(u)

get_count = json.loads(j.text)
data = get_count['aaData']

def remove_erroneous_chars(data_to_clean):
    chars_to_remove = {'comma': ',', 'dollar': '$'}
    
    print data_to_clean
    separate_change = data_to_clean.split('.')
    final_sal = separate_change[0].replace(',', '')
    final_sal = final_sal.replace('$', '')
    cleaned_data = int(final_sal.replace('.', ''))
    print cleaned_data
        
    return cleaned_data

def clean_data(data):
    print "data before clean"
    print data
    for i in data:
        print i[3]
        i[3] = remove_erroneous_chars(i[3])
        print i[3]
        
        print i[4]
        i[4] = remove_erroneous_chars(i[4])
        print i[4]
        
        i[8] = int(i[8])
    
    print "data after clean"
    print data
    return data
    

clean_data(data)

http://www.fedsdatacenter.com/federal-pay-rates/output.php?n=&a=&l=&o=&y=2015&sEcho=1&iColumns=9&sColumns=%2C%2C%2C%2C%2C%2C%2C%2C&iDisplayStart=0&iDisplayLength=10&mDataProp_0=0&bSortable_0=true&mDataProp_1=1&bSortable_1=true&mDataProp_2=2&bSortable_2=true&mDataProp_3=3&bSortable_3=true&mDataProp_4=4&bSortable_4=true&mDataProp_5=5&bSortable_5=true&mDataProp_6=6&bSortable_6=true&mDataProp_7=7&bSortable_7=true&mDataProp_8=8&bSortable_8=true&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&_=1469308589082
data before clean
[[u'A VANT,MAY ARDAH LINDA', u'14', u'GS', u'$118,057.00', u'$1,000.00', u'CITIZENSHIP AND IMMIGRATION SERVICES', u'WASHINGTON', u'MANAGEMENT AND PROGRAM ANALYSIS', u'2015'], [u"A'ANO,TRACIE M", u'12', u'GS', u'$70,192.00', u'$0.00', u'INDIAN HEALTH SERVICE', u'SELLS', u'MEDICAL TECHNOLOGIST', u'2015'], [u"A'MAR,Z TERESA", u'03', u'ZP', u'$76,633.00', u'$0.00', u'NAT OCEANIC AND ATMOSPHERIC ADMIN', u'SEATTLE', u'OPERATIONS RESEARCH', u'2015'], [u'A,MINH D', u'14', u'GS', u'$

[[u'A VANT,MAY ARDAH LINDA',
  u'14',
  u'GS',
  118057,
  1000,
  u'CITIZENSHIP AND IMMIGRATION SERVICES',
  u'WASHINGTON',
  u'MANAGEMENT AND PROGRAM ANALYSIS',
  2015],
 [u"A'ANO,TRACIE M",
  u'12',
  u'GS',
  70192,
  0,
  u'INDIAN HEALTH SERVICE',
  u'SELLS',
  u'MEDICAL TECHNOLOGIST',
  2015],
 [u"A'MAR,Z TERESA",
  u'03',
  u'ZP',
  76633,
  0,
  u'NAT OCEANIC AND ATMOSPHERIC ADMIN',
  u'SEATTLE',
  u'OPERATIONS RESEARCH',
  2015],
 [u'A,MINH D',
  u'14',
  u'GS',
  122571,
  5053,
  u'PATENT AND TRADEMARK OFFICE',
  u'ALEXANDRIA',
  u'PATENT EXAMINING',
  2015],
 [u'A,PHI DIEU TRAN',
  u'14',
  u'GS',
  134063,
  4002,
  u'PATENT AND TRADEMARK OFFICE',
  u'STAFFORD',
  u'PATENT EXAMINING',
  2015],
 [u'A-BOATENG,MARY',
  u'03',
  u'VN',
  90545,
  0,
  u'VETERANS HEALTH ADMINISTRATION',
  u'FAYETTEVILLE',
  u'NURSE',
  2015],
 [u'A-ONIKOYI,OLUREMI',
  u'01',
  u'VN',
  88812,
  0,
  u'VETERANS HEALTH ADMINISTRATION',
  u'PROVIDENCE',
  u'NURSE',
  2015],
 [u'A-ONIKOYI,OLUREMI',